In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
mnist_data, mnist_info = tfds.load('mnist', with_info=True, as_supervised=True)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.





Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.0. Subsequent calls will reuse this data.


In [ ]:
mnist_data

{'test': <DatasetV1Adapter shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>,
 'train': <DatasetV1Adapter shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>}

In [ ]:
mnist_train, mnist_test = mnist_data['train'], mnist_data['test']

In [ ]:
num_val_samples = tf.cast(0.1 * mnist_info.splits['train'].num_examples, tf.int64)
num_test_samples = tf.cast(mnist_info.splits['test'].num_examples, tf.int64)

In [ ]:
def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

In [ ]:
scaled_train_val_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [ ]:
BUFFER_SIZE = 1000

scaled_train_val_data = scaled_train_val_data.shuffle(BUFFER_SIZE)

In [ ]:
validation_data = scaled_train_val_data.take(num_val_samples)
train_data = scaled_train_val_data.skip(num_val_samples)

In [ ]:
BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_val_samples)
test_data = test_data.batch(num_test_samples)

In [ ]:
validation_input, validation_target = next(iter(validation_data))

In [ ]:
input_size = 784
hidden_layer_size = 100
output_size= 10

In [ ]:
model = tf.keras.Sequential([
                             tf.keras.layers.Flatten(input_shape=(28,28,1)),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1.0)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'], )

In [ ]:
model.fit(train_data, epochs=5, validation_data=(validation_input, validation_target))

Epoch 1/5
540/540 [==============================] - 14s 26ms/step - loss: 0.3366 - accuracy: 0.9052 - val_loss: 0.1806 - val_accuracy: 0.9475
Epoch 2/5
540/540 [==============================] - 14s 26ms/step - loss: 0.1433 - accuracy: 0.9584 - val_loss: 0.1445 - val_accuracy: 0.9568
Epoch 3/5
540/540 [==============================] - 14s 26ms/step - loss: 0.1001 - accuracy: 0.9705 - val_loss: 0.1211 - val_accuracy: 0.9647
Epoch 4/5
540/540 [==============================] - 14s 26ms/step - loss: 0.0742 - accuracy: 0.9781 - val_loss: 0.1017 - val_accuracy: 0.9712
Epoch 5/5
540/540 [==============================] - 14s 26ms/step - loss: 0.0583 - accuracy: 0.9829 - val_loss: 0.0994 - val_accuracy: 0.9722


In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 2ms/step - loss: 0.0856 - accuracy: 0.9741


In [ ]:
print('Test loss: {0:.2f} \nTest Accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09 
Test Accuracy: 97.41%
